In [146]:
def test_primalite(n):
    for i in range(2,int(n**(1/2))+1):
        if n%2 == 0:
            return False
        elif n%i ==0 :
            return False
    return True

def creation_q(bits):
    q = randint(2**(bits-1),2**bits)
    while test_primalite(q):
        q = randint(2**64,2**65)
    return q

def test_copremier(a,b):
    if xgcd(a,b)[0] == 1 :
        return True
    else :
        return False
    
def creation_cle(p):
    a = randint(p/2,p)
    return a

def double_add(P,d):
    while d != 1 :
        if d%2 == 0:
            P = 2*P
            d = int(d/2)
        else : 
            P = P + P
            d = d-1
    return P

def add(P,d):
    while d != 1 :
        P += P
        d -= 1
    return P

def point_gen(E):
    n = E.order()
    facteurs = factor(n)
    L = []
    for C in facteurs:
        L.append(C[0])
    r = max(L)
    h = n//r
    P = E.random_point()
    G = h*P
    return G,r


In [147]:
#EC Elgamal
def crea_cle(coef,p):
    a,b = coef[0]%p,coef[1]%p
    F = Zmod(p)
    E = EllipticCurve(F,coef)
    G,r = point_gen(E)
    
    d = randint(1,r-1)
    Q = d*G
    return (E,G,r,Q),d

def decrypt(sk,C):
    d = sk
    return C[1] - d*C[0]

def encrypt(pk,M):
    E,G,r,Q = pk
    k = randint(1,r-1)
    c1 = k*G
    c2 = k*Q + M
    return (c1,c2)

coef = [56,-47]
p = next_prime(creation_q(124))
print(f'Premier : {p}')
pk,sk = crea_cle(coef,p)
print(pk[0].order())
print(f'Clé privée : {sk}')

M = pk[0].random_point()
print(M)
C = encrypt(pk,M)
M1 = decrypt(sk,C)
print(M1)
print(M == M1)


Premier : 15513488053871810061774529268296229831
15513488053871810062770577470951525304
Clé privée : 1146148632563622025257981585937
(429013949491922682043811303245661932 : 15427248919078290310095037847241787521 : 1)
(429013949491922682043811303245661932 : 15427248919078290310095037847241787521 : 1)
True


In [149]:
def sign(pk,sk,m):
    E,G,r,Q = pk
    d = sk
    k = randint(1,r-1)
    R = k*G
    b = (int(R.xy()[0]))%r
    if b == 0 :
        return sign(pk,sk,m)
    inv_k = inverse_mod(k,int(r))
    s = (inv_k*(m+d*b))%r
    if s == 0 : 
        return sign(pk,sk,m)
    if not (1<= b <r and 1<=s<r):
        print('Pb dans la signature')
        return False
    return m,b,s

def verify_sign(pk,m,b,s):
    E,G,r,Q = pk
    inv_s = inverse_mod(s,r)
    u = (m*inv_s)%r
    v = (b*inv_s)%r
    print(b,int((u*G+v*Q)[0])%r)
    return int(b) == int((u*G+v*Q)[0])%r

m = 24768
ma, b, s = sign(pk,sk,m)
print(verify_sign(pk,ma,b,s))


376537473147221132845780355399 376537473147221132845780355399
True
